In [3]:
import subprocess
import pandas as pd
from pathlib import Path
import polars as pl
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)

ROOT=Path("/home/local/Downloads/parquet_files")
TRAIN_DIR=ROOT/"train"
TEST_DIR=ROOT/"test"


In [24]:
#read the files number
dirs=[TRAIN_DIR,TEST_DIR]
for dir in dirs:
    print(f"files under {dir}: {len(list(dir.glob('*parquet')))}")


#read the train files size
cmd=f'du {TRAIN_DIR}/* -mh|sort -rh '
res=subprocess.run(cmd,shell=True,stdout=subprocess.PIPE,text=True)
res_=res.stdout.split('\n')
res1=[line.split("\t") for line in res_ if line]
red=pd.DataFrame(res1,columns=["size","path"])
red["filename"]=red["path"].str.replace("train_","").apply(lambda x:Path(x).stem)

#read the test files size
cmd=f'du {TEST_DIR}/* -h|sort -rh '
res_test=subprocess.run(cmd,shell=True,stdout=subprocess.PIPE,text=True)
res_test_=res_test.stdout.split('\n')
res1_test=[line.split("\t") for line in res_test_ if line]
red_test=pd.DataFrame(res1_test,columns=["size","path"])
red_test["filename"]=red_test["path"].str.replace("test_","").apply(lambda x:Path(x).stem)

red_test.merge(red,on="filename",how="left",suffixes=("_test","_train")).sort_values(by="filename")

files under /home/local/Downloads/parquet_files/train: 32
files under /home/local/Downloads/parquet_files/test: 36


,size_test,path_test,filename,size_train,path_train
12,32K,/home/local/Downloads/parquet_files/test/test_applprev_1_0.parquet,applprev_1_0,103M,/home/local/Downloads/parquet_files/train/train_applprev_1_0.parquet
11,32K,/home/local/Downloads/parquet_files/test/test_applprev_1_1.parquet,applprev_1_1,70M,/home/local/Downloads/parquet_files/train/train_applprev_1_1.parquet
10,32K,/home/local/Downloads/parquet_files/test/test_applprev_1_2.parquet,applprev_1_2,NaN,NaN
33,8.0K,/home/local/Downloads/parquet_files/test/test_applprev_2.parquet,applprev_2,28M,/home/local/Downloads/parquet_files/train/train_applprev_2.parquet
35,4.0K,/home/local/Downloads/parquet_files/test/test_base.parquet,base,6.8M,/home/local/Downloads/parquet_files/train/train_base.parquet
7,60K,/home/local/Downloads/parquet_files/test/test_credit_bureau_a_1_0.parquet,credit_bureau_a_1_0,62M,/home/local/Downloads/parquet_files/train/train_credit_bureau_a_1_0.parquet
6,60K,/home/local/Downloads/parquet_files/test/test_credit_bureau_a_1_1.parquet,credit_bureau_a_1_1,175M,/home/local/Downloads/parquet_files/train/train_credit_bureau_a_1_1.parquet
5,60K,/home/local/Downloads/parquet_files/test/test_credit_bureau_a_1_2.parquet,credit_bureau_a_1_2,118M,/home/local/Downloads/parquet_files/train/train_credit_bureau_a_1_2.parquet
4,60K,/home/local/Downloads/parquet_files/test/test_credit_bureau_a_1_3.parquet,credit_bureau_a_1_3,69M,/home/local/Downloads/parquet_files/train/train_credit_bureau_a_1_3.parquet
3,60K,/home/local/Downloads/parquet_files/test/test_credit_bureau_a_1_4.parquet,credit_bureau_a_1_4,NaN,NaN


# Read train files

In [27]:
train_base=pd.read_parquet(TRAIN_DIR/"train_base.parquet")

train_base["date_decision"]=pd.to_datetime(train_base["date_decision"])
train_base["case_id"]=pd.to_numeric(train_base["case_id"],downcast="integer")
train_base["MONTH"]=pd.to_numeric(train_base["MONTH"],downcast="integer")
train_base["WEEK_NUM"]=pd.to_numeric(train_base["WEEK_NUM"],downcast="integer")
train_base["target"]=pd.to_numeric(train_base["target"],downcast="integer")
train_base["date_decision"].drop_duplicates().sort_values().diff().sum().days
len(train_base["date_decision"].unique())
train_base["date_decision"].dt.date.max()
train_base.target.value_counts()




,case_id,actualdpd_943P,annuity_853A,approvaldate_319D,byoccupationinc_3656910L,cancelreason_3545846M,childnum_21L,creationdate_885D,credacc_actualbalance_314A,credacc_credlmt_575A,credacc_maxhisbal_375A,credacc_minhisbal_90A,credacc_status_367L,credacc_transactions_402L,credamount_590A,credtype_587L,currdebt_94A,dateactivated_425D,district_544M,downpmt_134A,dtlastpmt_581D,dtlastpmtallstes_3545839D,education_1138M,employedfrom_700D,familystate_726L,firstnonzeroinstldate_307D,inittransactioncode_279L,isbidproduct_390L,isdebitcard_527L,mainoccupationinc_437A,maxdpdtolerance_577P,num_group1,outstandingdebt_522A,pmtnum_8L,postype_4733339M,profession_152M,rejectreason_755M,rejectreasonclient_4145042M,revolvingaccount_394A,status_219L,tenor_203L
0,2,0.0,640.2,None,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,NaN,NaN,None,NaN,10000.0,CAL,NaN,None,P136_108_173,0.0,None,None,P97_36_170,2010-02-15,SINGLE,2013-05-04,CASH,False,None,8200.0,NaN,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,24.0
1,2,0.0,1682.4,None,NaN,a55475b1,0.0,2013-04-03,NaN,0.0,NaN,NaN,None,NaN,16000.0,CAL,NaN,None,P136_108_173,0.0,None,None,P97_36_170,2010-02-15,SINGLE,2013-05-04,CASH,False,None,8200.0,NaN,1,NaN,12.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,D,12.0
2,3,0.0,6140.0,None,NaN,P94_109_143,NaN,2019-01-07,NaN,0.0,NaN,NaN,None,NaN,59999.8,CAL,NaN,None,P131_33_167,0.0,None,None,P97_36_170,2018-05-15,MARRIED,2019-02-07,CASH,False,None,11000.0,NaN,0,NaN,12.0,a55475b1,a55475b1,P94_109_143,a55475b1,NaN,D,12.0
3,4,0.0,2556.6,None,NaN,P24_27_36,NaN,2019-01-08,NaN,0.0,NaN,NaN,None,NaN,40000.0,CAL,NaN,None,P194_82_174,0.0,None,None,a55475b1,None,None,2019-02-08,CASH,False,None,16000.0,NaN,0,NaN,24.0,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,24.0
4,5,0.0,NaN,None,NaN,P85_114_140,NaN,2019-01-16,NaN,NaN,NaN,NaN,None,NaN,NaN,None,NaN,None,P54_133_26,NaN,None,None,a55475b1,None,None,None,None,False,None,62000.0,NaN,0,NaN,NaN,a55475b1,a55475b1,a55475b1,a55475b1,NaN,T,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3887679,2651092,0.0,3000.0,2019-12-30,NaN,a55475b1,NaN,2019-12-30,53300.0,53300.0,0.0,0.0,CL,0.0,53998.0,COL,53998.0,None,P147_21_170,0.0,None,None,a55475b1,None,None,2020-01-30,POS,False,None,60000.0,NaN,0,53998.0,18.0,P177_117_192,a55475b1,a55475b1,a55475b1,NaN,N,18.0
3887680,2651092,0.0,3119.2,2012-12-12,25000.0,a55475b1,1.0,2012-12-12,NaN,0.0,NaN,NaN,None,NaN,25740.0,COL,0.0,2012-12-20,P147_21_170,0.0,None,None,P97_36_170,2004-10-15,MARRIED,2013-01-11,POS,False,None,10000.0,0.0,7,0.0,12.0,P149_40_170,a55475b1,a55475b1,a55475b1,NaN,K,12.0
3887681,2651092,0.0,4366.0,2017-11-09,NaN,a55475b1,NaN,2017-11-09,NaN,0.0,NaN,NaN,None,NaN,19638.0,COL,0.0,2017-11-15,P147_21_170,0.0,2018-04-03,2018-04-03,a55475b1,None,LIVING_WITH_PARTNER,2017-12-10,POS,False,None,40000.0,0.0,4,0.0,5.0,P60_146_156,a55475b1,a55475b1,a55475b1,NaN,K,5.0
3887682,2651092,0.0,4496.6,2019-02-17,NaN,a55475b1,NaN,2019-02-17,NaN,0.0,NaN,NaN,None,NaN,20840.0,COL,0.0,2019-02-18,P147_21_170,0.0,2019-07-31,2019-07-31,a55475b1,None,None,2019-03-17,POS,False,None,60000.0,0.0,2,0.0,6.0,P149_40_170,a55475b1,a55475b1,a55475b1,NaN,K,6.0


In [74]:
#def read_files(path,depth=None):


data_store={"df_base":pl.read_parquet(TRAIN_DIR/"train_base.parquet"),
            "depth_0":pl.read_parquet(TRAIN_DIR/"train_static_cb_0.parquet")}
data_store


{'df_base': shape: (1_526_659, 5)
 ┌─────────┬───────────────┬────────┬──────────┬────────┐
 │ case_id ┆ date_decision ┆ MONTH  ┆ WEEK_NUM ┆ target │
 │ ---     ┆ ---           ┆ ---    ┆ ---      ┆ ---    │
 │ i64     ┆ str           ┆ i64    ┆ i64      ┆ i64    │
 ╞═════════╪═══════════════╪════════╪══════════╪════════╡
 │ 0       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 1       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 2       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 0      │
 │ 3       ┆ 2019-01-03    ┆ 201901 ┆ 0        ┆ 0      │
 │ 4       ┆ 2019-01-04    ┆ 201901 ┆ 0        ┆ 1      │
 │ …       ┆ …             ┆ …      ┆ …        ┆ …      │
 │ 2703450 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703451 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703452 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703453 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 │ 2703454 ┆ 2020-10-05    ┆ 202010 ┆ 91       ┆ 0      │
 └─────────┴───────────────┴────────┴─